In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid19-data/by_gender.xlsx
/kaggle/input/covid19-data/humdata_epidemic_risks.xlsx
/kaggle/input/covid19-data/owid-covid-data.csv
/kaggle/input/covid19-data/COVID19 inpatient cases data.xls
/kaggle/input/covid19-data/timeseries_confirmed_deaths.csv
/kaggle/input/covid19-data/humdata_govt_response_investments.csv
/kaggle/input/covid19-data/Provisional_Death_Counts_for_Coronavirus_Disease__COVID-19_.csv
/kaggle/input/covid19-data/days_to_double_cases.csv
/kaggle/input/covid19-data/time_series_covid19_recovered_global.csv
/kaggle/input/covid19-data/over65_totalpopulation_bycountry.csv
/kaggle/input/covid19-data/acaps_Global_PublicHealth_SocialMeasures.csv
/kaggle/input/covid19-data/timeseries_confirmed_cases_global.csv
/kaggle/input/measures-4/acaps_Global_PublicHealth_SocialMeasures.csv


In [2]:
import pandas as pd
import numpy as np
import gc
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.gridspec as gridspec

gender = pd.read_excel('../input/covid19-data/by_gender.xlsx')
inpatient = pd.read_excel('../input/covid19-data/COVID19 inpatient cases data.xls')
confirmed = pd.read_csv('../input/covid19-data/timeseries_confirmed_cases_global.csv')
deaths = pd.read_csv('../input/covid19-data/timeseries_confirmed_deaths.csv')
recovered = pd.read_csv('../input/covid19-data/time_series_covid19_recovered_global.csv')
measures = pd.read_csv('../input/measures-4/acaps_Global_PublicHealth_SocialMeasures.csv')
pd.set_option('display.max_columns', None)
print(inpatient.shape, gender.shape, confirmed.shape, deaths.shape,
     recovered.shape, measures.shape)

(74, 193) (78, 14) (266, 115) (266, 115) (252, 115) (27554, 8)


### This notebook will be divided into 3 parts:

#### Part 1: Combine Datasets and Cleaning (#1-#25)
#### Part 2: Data Preprocessing (#25-#39)
#### Part 3: Data Analysis (#40-#69)



### Part-1: Combine Datasets and Cleaning (#1-#25)

Explore gender dataset

In [3]:
#1
gender.head()

,Country,Sex-disaggregated?,Date,Total cases,Cases (% male),Cases (% female),Total deaths,deaths (% male),deaths (% female),Proportion of deaths in confirmed cases (total),Proportion of deaths in confirmed cases (male),Proportion of deaths in confirmed cases (female),Proportion of deaths in confirmed cases (Male:female ratio),Sources
0,Australia,Yes,5-May,6775.0,0.503000,0.497000,96,0.573000,0.427000,0.014170,0.016134,0.012181,1.324543,http://www9.health.gov.au/cda/source/rpt_5.cfm
1,Austria,Yes,2020-05-05 00:00:00,15585.0,0.490000,0.510000,606,0.570000,0.430000,0.038884,0.045175,0.032838,1.375668,https://www.sozialministerium.at/Informationen...
2,Belgium,Yes,2020-05-05 00:00:00,50320.0,0.366000,0.634000,5647,0.507880,0.492120,0.112222,0.155785,0.087089,1.788808,https://epistat.wiv-isp.be/Covid/covid-19.html
3,Brazil,Partial,2020-04-26 00:00:00,61888.0,NaN,NaN,3611,0.593187,0.406813,NaN,NaN,NaN,NaN,https://portalarquivos.saude.gov.br/images/pdf...
4,Canada,Yes,2020-05-05 00:00:00,32531.0,0.448833,0.551167,1590,0.473585,0.526415,0.048876,0.051572,0.046682,1.104758,https://www.canada.ca/content/dam/phac-aspc/do...


#### If Male has higher % of cases, set as Male, or otherwise

In [4]:
#2
gender['Gender'] = gender['Cases (% male)'] - gender['Cases (% female)']
gender['Gender'] = np.where(gender['Gender']> 0.00, 'Male', 'Female')

In [5]:
#3
gender = gender[['Country', 'Gender', 'Cases (% male)', 'Cases (% female)',
                 ]]
print(gender.shape)

(78, 4)


Rename Country items for Consistency when combined with other datasets

In [6]:
#4
gender['Country'] = gender['Country'].str.replace('The Netherlands', 'Netherlands')
gender['Country'] = gender['Country'].str.replace('Republic of Ireland', 'Ireland')
gender['Country'] = gender['Country'].str.replace('Northern Ireland', 'Ireland')
gender['Country'] = gender['Country'].str.replace('Kazakstan', 'Kazakhstan')
gender['Country'] = gender['Country'].str.replace('England and Wales', 'United Kingdom')
gender['Country'] = gender['Country'].str.replace('England and Wales', 'United Kingdom')

Let' explore covid-19 inpatient dataset

In [7]:
#5
inpatient.head()

,Group,"Course of lopinavir-ritonavir (LR, 400 mg and 100mg) bid","Course of arbidol (Ar, 200mg) tid",Gender,"Age, year",Clinical classification,"Days from onset of symptoms to hospital admission, d",Comorbidities,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Signs and symptoms,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Clinical outcome,"Hospital stay, days",COVID-19 RNA clearance,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,Unnamed: 122,Unnamed: 123,Unnamed: 124,Unnamed: 125,Unnamed: 126,Unnamed: 127,Unnamed: 128,Unnamed: 129,Unnamed: 130,Unnamed: 131,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140,Unnamed: 141,Unnamed: 142,Unnamed: 143,Unnamed: 144,Unnamed: 145,Unnamed: 146,Unnamed: 147,Unnamed: 148,Unnamed: 149,Unnamed: 150,Unnamed: 151,Unnamed: 152,Unnamed: 153,Unnamed: 154,Unnamed: 155,"The time of nucleic acid turning negative, days",Chest CT findings at discharge,Days after hospitalization,"White Blood Cell Count, ×109/L","Neutrophil count, ×109/L","Lymphocyte count, ×109/L","Monocyte count, ×109/L","CRP, mg/L","PCT, ng/mL",Days after hospitalization.1,"White Blood Cell Count, ×109/L.1","Neutrophil count, ×109/L.1","Lymphocyte count, ×109/L.1","Monocyte count, ×109/L.1","CRP, mg/L.1","PCT, ng/mL.1",Days after hospitalization.2,"White Blood Cell Count, ×109/L.2","Neutrophil count, ×109/L.2","Lymphocyte count, ×109/L.2","Monocyte count, ×109/L.2","CRP, mg/L.2","PCT, ng/mL.2",Days after hospitalization.3,"White Blood Cell Count, ×109/L.3","Neutrophil count, ×109/L.3","Lymphocyte count, ×109/L.3","Monocyte count, ×109/L.3","CRP, mg/L.3","PCT, ng/mL.3",Days after hospitalization.4,"White Blood Cell Count, ×109/L.4","Neutrophil count, ×109/L.4","Lymphocyte count, ×109/L.4","Monocyte count, ×109/L.4","CRP, mg/L.4","PCT, ng/mL.4"
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No comorbidity,Cardiovascular and\ncerebrovascular disease,Endocrine system disease,Malignant tumor,Respiratory system disease,Digestive system disease,Renal disease,Liver disease,Fever,"Disappearance time, days",Cough,"Disappearance time, days",Chest tightness,"Disappearance time, days",Fatigue,"Disappearance time, days",Diarrhea,"Disappearance time, days",Others,"Disappearance time, days",NaN,NaN,NaN,入院日期：,出院日期：,住院时间/天,其他,体征消失时间,疗程,克力芝(开嘱日期),克力芝(停嘱日期),疗程/天,阿比朵尔(开嘱日期),阿比朵尔(停嘱日期),疗程/天,莲花清瘟,免疫球蛋白,甲泼尼龙,中药,入院1-2天,住院天数,白细胞,中性粒细胞,淋巴细胞,单核细胞,ALT,AST,GGT,TBIL,DBIL,IBIL,ALB,Cr,尿素氮,D二聚体,凝血酶原时间PT,纤维蛋白原,CRP,PCT,BNP,LDH,入院3-5天,住院天数,白细胞,中性粒细胞,淋巴细胞,单核细胞,ALT,AST,GGT,TBIL,DBIL,IBIL,ALB,Cr,尿素氮,D二聚体,凝血酶原时间PT,纤维蛋白原,CRP,PCT,BNP,LDH,入院6-9,住院天数,白细胞,中性粒细胞,淋巴细胞,单核细胞,ALT,AST,GGT,TBIL,DBIL,IBIL,ALB,Cr,尿素氮,D二聚体,凝血酶原时间PT,纤维蛋白原,CRP,PCT,BNP,LDH,入院10-14,住院天数,白细胞,中性粒细胞,淋巴细胞,单核细胞,ALT,AST,GGT,TBIL,DBIL,IBIL,ALB,Cr,尿素氮,D二聚体,凝血酶原时间PT,纤维蛋白原,CRP,PCT,BNP,LDH,入院＞

In [8]:
#6
inpatient = inpatient[['Group', 'Course of lopinavir-ritonavir (LR, 400 mg and 100mg) bid', 'Course of arbidol (Ar, 200mg) tid',
 'Gender', 'Age, year', 'Clinical classification', 'Days from onset of symptoms to hospital admission, d',
 'Comorbidities', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
 'Unnamed: 13', 'Unnamed: 14', 'Signs and symptoms', 'Unnamed: 17', 'Unnamed: 19',
 'Unnamed: 21', 'Unnamed: 23', 'Clinical outcome', 'Hospital stay, days', 'COVID-19 RNA clearance',
 'Unnamed: 30', 'Unnamed: 31', 'The time of nucleic acid turning negative, days', 'Chest CT findings at discharge',
 'Days after hospitalization', 'White Blood Cell Count, ×109/L', 'Neutrophil count, ×109/L',
 'Lymphocyte count, ×109/L', 'Monocyte count, ×109/L', 'CRP, mg/L', 'PCT, ng/mL']]
print(inpatient.shape)

(74, 34)


Row# 0 provides Label for Unnamed columns; Rename the columns and Drop Row# 0

In [9]:
#7
inpatient.rename(columns={'Unnamed: 8': 'Cardiovascular and\ncerebrovascular disease', 
                          'Unnamed: 9': 'Endocrine system disease',
                          'Unnamed: 10': 'Malignant tumor', 'Unnamed: 11': 'Respiratory system disease',
                          'Unnamed: 12': 'Digestive system disease', 'Unnamed: 13': 'Chronic kidney disease',
                          'Unnamed: 14': 'Liver disease', 'Unnamed: 17': 'Cough',
                          'Unnamed: 19': 'Chest tightness', 'Unnamed: 21': 'Fatigue',
                          'Unnamed: 23': 'Diarrhea', 'Unnamed: 30': 'Entered Hospital',
                          'Unnamed: 31': 'Discharged/Left',
                        }, inplace=True)

In [10]:
#8
inpatient = inpatient.loc[1:,:]
print(inpatient.shape)

(73, 34)


-> Not using original Hospital stay, days (which has huge Null values)

-> Instead, calculate day difference between Entered Hospital and 

Discharged/Left, to arrive at 'Days at Hospital' (Result: Zero Null Values)

-> Drop unnecessary columns

In [11]:
#9
inpatient['Entered Hospital'] = pd.to_datetime(inpatient['Entered Hospital'])
inpatient['Discharged/Left'] = pd.to_datetime(inpatient['Discharged/Left'])

inpatient['Days at Hospital'] = inpatient['Discharged/Left'] - inpatient['Entered Hospital']
inpatient['Days at Hospital'] = inpatient['Days at Hospital'].astype('str').str.slice(stop=2)
inpatient['Days at Hospital'] = inpatient['Days at Hospital'].astype('int32')
inpatient.drop([
'Entered Hospital',
'Discharged/Left', 
'Group', 
'Hospital stay, days', 
#'The time of nucleic acid turning negative, days',
'Chest CT findings at discharge',
'Days after hospitalization',
#'White Blood Cell Count, ×109/L',
#'Neutrophil count, ×109/L',
#'Lymphocyte count, ×109/L',
#'Monocyte count, ×109/L',
#'CRP, mg/L',
#'PCT, ng/mL',
], axis=1, inplace=True)

-> Combine gender and inpatient to build relationship between gender and patient conditions

-> New dataframe: gender_inpatient

In [12]:
#10
gender_inpatient = pd.merge(gender, inpatient, on='Gender', how='left')
print(gender_inpatient.shape)
gender_inpatient.drop(['Course of arbidol (Ar, 200mg) tid'], axis=1, inplace=True)
#Cases (% male) #Cases (% female) 

(2831, 32)


Rename to Higher_Pct_Gender for better understanding 

(if Male has higher %of cases -> Male, else Female)

In [13]:
#11
gender_inpatient.rename(columns={'Gender': 'Higher_Pct_Gender'}, inplace=True)

Explore gender_inpatient dataframe

In [14]:
#12
gender_inpatient.head()

,Country,Higher_Pct_Gender,Cases (% male),Cases (% female),"Course of lopinavir-ritonavir (LR, 400 mg and 100mg) bid","Age, year",Clinical classification,"Days from onset of symptoms to hospital admission, d",Comorbidities,Cardiovascular and\ncerebrovascular disease,Endocrine system disease,Malignant tumor,Respiratory system disease,Digestive system disease,Chronic kidney disease,Liver disease,Signs and symptoms,Cough,Chest tightness,Fatigue,Diarrhea,Clinical outcome,COVID-19 RNA clearance,"The time of nucleic acid turning negative, days","White Blood Cell Count, ×109/L","Neutrophil count, ×109/L","Lymphocyte count, ×109/L","Monocyte count, ×109/L","CRP, mg/L","PCT, ng/mL",Days at Hospital
0,Australia,Male,0.503,0.497,6.0,47.0,Heavy,15.0,Y,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,"Aggravated,turn to ICU",N,NaN,1.70,0.78,0.79,0.12,8.4,0.04,6
1,Australia,Male,0.503,0.497,4.0,58.0,Heavy,1.0,N,N,Y,N,N,N,N,N,N,N,N,N,N,"Aggravated,transfer to superior hospital",N,NaN,3.20,1.70,1.30,0.22,16.1,0.04,4
2,Australia,Male,0.503,0.497,5.0,54.0,Ordinary,10.0,Y,N,N,N,N,N,N,N,Y,Y,Y,N,N,Improved and discharge,Y,4.0,3.03,2.41,0.54,0.07,6.4,0.04,5
3,Australia,Male,0.503,0.497,5.0,21.0,Ordinary,40.0,Y,N,N,N,N,N,N,N,N,Y,N,N,N,Improved and discharge,Y,2.0,5.50,3.37,1.70,0.32,4.4,0.04,5
4,Australia,Male,0.503,0.497,6.0,62.0,Ordinary,2.0,Y,N,N,N,N,N,N,N,N,N,N,N,N,Improved and discharge,Y,3.0,NaN,NaN,NaN,NaN,NaN,NaN,6


In [15]:
#13
#confirmed.head()

In [16]:
#14
#deaths.head()

In [17]:
#15
recovered.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,5,5,10,10,10,15,18,18,29,32,32,32,32,32,40,43,54,99,112,131,135,150,166,179,188,188,207,220,228,252,260,310,331,345,397,421,458,468,472,502,558,558
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,10,17,17,31,31,33,44,52,67,76,89,99,104,116,131,154,165,182,197,217,232,248,251,277,283,302,314,327,345,356,385,394,403,410,422,431,455,470,488,519,531,543,570,595,605,620,627,650,654
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,12,12,12,12,12,32,32,32,65,65,24,65,29,29,31,31,37,46,61,61,62,90,90,90,113,237,347,405,460,591,601,691,708,783,846,894,1047,1099,1152,1204,1355,1408,1479,1508,1558,1651,1702,1779,1821,1872,1936,1998,2067,2197,2323,2467,2546,2678,2841
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,10,10,10,10,16,21,26,31,39,52,58,71,71,128,128,128,169,169,191,205,235,248,282,309,333,344,344,344,385,398,423,468,468,472,493,499,514,521,526,537,545,550,550
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,2,2,2,2,2,2,2,4,4,4,5,5,5,5,6,6,6,6,6,6,6,6,6,6,6,7,7,11,11,11,11,11,11,11,11,13,13,13


We will look at recovery from 4/1/20 to 5/11/20

In [18]:
#16
recovered = recovered[['Country/Region', 
                                         '4/1/20', '4/2/20', '4/3/20', '4/4/20', '4/5/20', '4/6/20',
                                         '4/7/20', '4/8/20', '4/9/20', '4/10/20', '4/11/20', '4/12/20',
                                         '4/13/20', '4/14/20', '4/15/20', '4/16/20', '4/17/20', '4/18/20',
                                         '4/19/20', '4/20/20', '4/21/20', '4/22/20', '4/23/20', '4/24/20',
                                         '4/25/20', '4/26/20', '4/27/20', '4/28/20', '4/29/20', '4/30/20',
                                         '5/1/20', '5/2/20', '5/3/20', '5/4/20', '5/5/20', '5/6/20',
                                         '5/7/20', '5/8/20', '5/9/20', '5/10/20', '5/11/20', 
                                        ]]
recovered.rename(columns={'Country/Region': 'Country'}, inplace=True)
print(recovered.shape)

(252, 42)


Again, rename Country items for Consistency when combined with other datasets

In [19]:
#17
recovered['Country'] = recovered['Country'].str.replace('Korea, South', 'South Korea')
recovered['Country'] = recovered['Country'].str.replace('Czechia', 'Czech Republic')
recovered['Country'] = recovered['Country'].str.replace('US', 'USA')
recovered['Country'] = recovered['Country'].str.replace('United Arab Emirates', 'UAE')
recovered['Country'] = recovered['Country'].str.replace('Burma', 'Myanmar')
recovered['Country'] = recovered['Country'].str.replace('Timor-Leste', 'Timor')
recovered['Country'] = recovered['Country'].str.replace('Taiwan*', 'Taiwan')

Add-in 'recovered' in front of each date for clarification to rename Time columns

In [20]:
#18
recovered.rename(columns={
    '4/1/20':'recovered_4/1/20', '4/2/20':'recovered_4/2/20', '4/3/20':'recovered_4/3/20', 
    '4/4/20':'recovered_4/4/20', '4/5/20':'recovered_4/5/20', '4/6/20':'recovered_4/6/20',
    '4/7/20':'recovered_4/7/20', '4/8/20':'recovered_4/8/20', '4/9/20':'recovered_4/9/20', 
    '4/10/20':'recovered_4/10/20', '4/11/20':'recovered_4/11/20', '4/12/20':'recovered_4/12/20',
    '4/13/20':'recovered_4/13/20', '4/14/20':'recovered_4/14/20', '4/15/20':'recovered_4/15/20', 
    '4/16/20':'recovered_4/16/20', '4/17/20':'recovered_4/17/20', '4/18/20':'recovered_4/18/20',
    '4/19/20':'recovered_4/19/20', '4/20/20':'recovered_4/20/20', '4/21/20':'recovered_4/21/20', 
    '4/22/20':'recovered_4/22/20', '4/23/20':'recovered_4/23/20', '4/24/20':'recovered_4/24/20',
    '4/25/20':'recovered_4/25/20', '4/26/20':'recovered_4/26/20', '4/27/20':'recovered_4/27/20',
    '4/28/20':'recovered_4/28/20', '4/29/20':'recovered_4/29/20', '4/30/20':'recovered_4/30/20',
    '5/1/20':'recovered_5/1/20', '5/2/20':'recovered_5/2/20', '5/3/20':'recovered_5/3/20',
    '5/4/20':'recovered_5/4/20', '5/5/20':'recovered_5/5/20', '5/6/20':'recovered_5/6/20',
    '5/7/20':'recovered_5/7/20', '5/8/20':'recovered_5/8/20','5/9/20':'recovered_5/9/20',
    '5/10/20':'recovered_5/10/20', '5/11/20':'Latestrecovered_5/11/20'}, inplace=True)

Let's look at Protective Measures/Factors dataset

In [21]:
#19
measures.head()

,country_territory_area,admin_level,prov_category,prov_measure,who_subcategory,who_measure,targeted,non_compliance
0,Afghanistan,national,Public health measures,Health screenings in airports and border cross...,NaN,Entry screening and isolation or quarantine,No,NaN
1,Afghanistan,other,Public health measures,Introduction of isolation and quarantine policies,Detecting and isolating cases,Isolation,No,NaN
2,Afghanistan,national,Public health measures,Awareness campaigns,Communications and engagement,General public awareness campaigns,No,NaN
3,Afghanistan,national,Governance and socio-economic measures,Emergency administrative structures activated ...,NaN,Financial packages,No,NaN
4,Afghanistan,national,Social distancing,Limit public gatherings,"Gatherings, businesses and services",NaN,No,NaN


Here, rename columns for Readability

In [22]:
#20
measures.rename(columns={'country_territory_area': 'Country',
                         'prov_category': 'protection_category',
                         'prov_measure': 'protection_measure',
                         'who_subcategory': 'who_category',
                         'admin_level': 'protective_measures_admin_level'
                         }, inplace=True)

-> 'protective_measure' is similar to 'protection_category'

-> 'who_measure' is similar to 'who_category'
so, drop both columns

In [23]:
#21
#Drop similar columns
measures.drop(['protection_measure','who_measure'], axis=1, inplace=True)

Again, rename Country items for Consistency when combined with other datasets

In [24]:
#22
measures['Country'] = measures['Country'].str.replace('Democratic Republic Of The Congo', 'Democratic Republic Of Congo')
measures['Country'] = measures['Country'].str.replace("Democratic People's Republic Of Korea", 'North Korea')
measures['Country'] = measures['Country'].str.replace("Bolivia, Plurinational State Of", 'Bolivia')
measures['Country'] = measures['Country'].str.replace("Iran, Islamic Republic Of", 'Iran')
measures['Country'] = measures['Country'].str.replace("Lao People's Democratic Republic", 'Laos')
measures['Country'] = measures['Country'].str.replace('Republic Of Moldova', "Moldova")
measures['Country'] = measures['Country'].str.replace("Republic Of Korea", 'South Korea')
measures['Country'] = measures['Country'].str.replace("Syrian Arab Republic", 'Syria')
measures['Country'] = measures['Country'].str.replace("Timor-Leste", 'Timor')
measures['Country'] = measures['Country'].str.replace('United Republic Of Tanzania', "Tanzania")
measures['Country'] = measures['Country'].str.replace("United Kingdom Of Great Britain And Northern Ireland", 'UK')
measures['Country'] = measures['Country'].str.replace("United Arab Emirates", 'UAE')
measures['Country'] = measures['Country'].str.replace("Venezuela, Bolivarian Republic Of", 'Venezuela')
measures = measures[measures['Country']!='Northern Mariana Islands, Commonwealth Of The']
measures = measures[measures['Country']!='Occupied Palestinian Territory, Including East Jerusalem']

2 steps of Merging:

-> 1st Merge: Combine gender_inpatient and recovered into new dataframe to build relationship between Recovery and Patient Conditions

-> New dataframe: recover_inpatient

In [25]:
#23
recover_inpatient = pd.merge(gender_inpatient, recovered, on='Country', how='left')
print(recover_inpatient.shape)

(4951, 72)


2nd Merge: Combine recover_inpatient from 1st with measures to arrive at final dataframe

-> Final dataframe: recover_inpatient_measures

a) Represents: Protective Factors vs Patient Conditions vs Recovery

b) "What Protective Factors will improve Recovery based on Patient Conditions"

In [26]:
#24
recover_inpatient_measures = pd.merge(recover_inpatient, measures, on='Country', how='inner')
print(recover_inpatient_measures.shape) #shape = 2830777, 77

(2830777, 77)


### Part-2: Data Preprocessing (#25-#39)

Now, we have final dataframe to answer research question, so let's save it for reusability

-> Final dataframe: recover_inpatient_measures includes data for

a) Protective Factors vs Patient Conditions vs Recovery

b) "What Protective Factors will improve Recovery based on Patient Conditions"

In [27]:
#25
recover_inpatient_measures.to_csv('recover_inpatient_measures.csv', index=False)
recover_inpatient_measures.to_pickle('recover_inpatient_measures.pkl')

Delete unnecessary dataframes to save memory

In [28]:
#26
del gender
del inpatient
del gender_inpatient
del confirmed
del deaths
del recovered
del measures
gc.collect()

348

Let's check for patients with Comorbidities

-> Seems like number of patients w/ Comorbidities (a Major Risk Factor) is HIGHER than those w/o

-> Data Analysis part will explain how and what Protective Factors will help Raise recovery for patients with comorbidites

In [29]:
#27
recover_inpatient_measures['Comorbidities'].value_counts().sort_values(ascending=False)

Y    1523936
N    1306841
Name: Comorbidities, dtype: int64

'Clinical outcome' represents whether patients

1) have Improved and are Discharged from hospitals (recovery after hospitalization)

2) Died

3) Transfer to ICU/other superior hospitals (recovery after ICU admission)

In [30]:
#28
recover_inpatient_measures['Clinical outcome'].value_counts().sort_index(ascending=False)

Improved and discharge                      2677031
Died                                          27048
Aggravated,turn to ICU                        63349
Aggravated,transfer to superior hospital      63349
Name: Clinical outcome, dtype: int64

Let's do some value mapping for 'targeted' feature

specific group -> could be based on Age, Gender, Health Prevalence,...

In [31]:
#29
recover_inpatient_measures['targeted'].fillna('No', inplace=True)
recover_inpatient_measures['targeted'] = recover_inpatient_measures['targeted'].map({
'No': 'Not_targeted',
'Yes': 'targeted',
'General': 'Not_targeted',
'Targeted': 'targeted',
})
recover_inpatient_measures['Age, year']=recover_inpatient_measures['Age, year'].astype('int32')

The Next 2 blocks of codes take care of Null values in the following features

In [32]:
#30
recover_inpatient_measures['The time of nucleic acid turning negative, days'].fillna(13.0, inplace=True)
recover_inpatient_measures['The time of nucleic acid turning negative, days'] = recover_inpatient_measures['The time of nucleic acid turning negative, days'].astype('int32')

In [33]:
#31
#fill in Null values with Mode 
recover_inpatient_measures['protection_category'].fillna('Quarantine / Monitoring', inplace=True)
recover_inpatient_measures['who_category'].fillna('Communications and engagement', inplace=True)

Feature Mapping for 'protection_category'

Group similar values into respective categories

Name the new column as 'protective_factors', Drop original 'protection_category'

In [34]:
#32
factors = []
for row in recover_inpatient_measures['protection_category']:
    
    if row == 'Mass Gathering Cancellation':
        factors.append('Public health measures')
    elif row == 'Healthcare and public health capacity':
        factors.append('Public health measures')
    elif row == 'Ppe':
        factors.append('Public health measures')
    
    elif row == 'Case identification, contact tracing and related measures':
        factors.append('Detecting and isolating cases')
    elif row == 'Risk communication':
        factors.append('Detecting and isolating cases')
    
    elif row == 'Social distancing':
        factors.append('Quarantine / Monitoring')
    elif row == 'Movement restrictions':
        factors.append('Quarantine / Monitoring')
    elif row == 'Other':
        factors.append('Quarantine / Monitoring')
    
    elif row == 'Research Advancement':
        factors.append('Research and Vaccine Advancement')
    elif row == 'Vaccine/Mcm Deployed':
        factors.append('Research and Vaccine Advancement')
    
    elif row == 'Travel Ban':
        factors.append('Travel_Ban_Advisory_Restriction')
    elif row == 'Travel Advisory':
        factors.append('Travel_Ban_Advisory_Restriction')
    elif row == 'Travel restriction':
        factors.append('Travel_Ban_Advisory_Restriction')
    elif row == 'Repatriation':
        factors.append('Airport Screening: Entry')

    else:
        factors.append(row)
        
recover_inpatient_measures['protective_factors'] = factors
recover_inpatient_measures.drop(['protection_category'], axis=1, inplace=True)

Same here as well, Feature Mapping for 'who_category' (WHO)

Group similar values into respective categories

Name the new column as 'who_protective_factors', Drop original 'who_category'

In [35]:
#33
who_factors = []
for row in recover_inpatient_measures['who_category']:
    
    if row == 'Tracing and quarantining contacts':
        who_factors.append('Detecting and isolating cases')
    elif row == 'Risk communication':
        who_factors.append('Detecting and isolating cases')
    
    elif row == 'unknown':
        who_factors.append('Communications and engagement')
    
    elif row == 'Gatherings, businesses and services':
        who_factors.append('Public health measures')
    elif row == 'Offices, businesses, institutions and operations':
        who_factors.append('Public health measures') 
    elif row == 'School measures':
        who_factors.append('Public health measures') 
        
    else:
        who_factors.append(row)
        
recover_inpatient_measures['who_protective_factors'] = who_factors
recover_inpatient_measures.drop(['who_category'], axis=1, inplace=True)

Feature Mapping for 'Days at Hospital'

Group into 5 categories -> stay for 10-days, 2-week, 3-week, 25-days, morethan 25-days

Each will indirectly represent severity of patients' conditions

Name new column as 'Hospital_Days_Group'

In [36]:
#34
hdays_group = []
for row in recover_inpatient_measures['Days at Hospital']:
    if row <= 10:
        hdays_group.append('10-days')
    elif row > 10 and row <= 15:
        hdays_group.append('2-week')
    elif row > 15 and row <= 20:
        hdays_group.append('3-week')
    elif row > 20 and row <= 25:
        hdays_group.append('25-days')
    else:
        hdays_group.append('morethan_25days')
recover_inpatient_measures['Hospital_Days_Group'] = hdays_group

Group 'protective_factors' based on frequency counting

-> Which protective factors are more frequently applied than others

Name the new column as 'top6_protective_factors_1'

Quarantine / Monitoring & Public health measures top the list

We will use this feature to analyze how it affects recovery

In [37]:
#35
top_6_1 = []
for row in recover_inpatient_measures['protective_factors']:
    
    if row == 'Quarantine / Monitoring':
        top_6_1.append('Quarantine / Monitoring')
   
    elif row == 'Public health measures':
        top_6_1.append('Public health measures')
    
    elif row == 'Research and Vaccine Advancement':
        top_6_1.append('Research and Vaccine Advancement')
    
    elif row == 'Travel_Ban_Advisory_Restriction':
        top_6_1.append('Travel_Ban_Advisory_Restriction')
    
    elif row == 'Education Campaign':
        top_6_1.append('Education Campaign') 
    
    elif row == 'Detecting and isolating cases':
        top_6_1.append('Detecting and isolating cases') 
        
    else:
        top_6_1.append('Social measures')
        
recover_inpatient_measures['top6_protective_factors_1'] = top_6_1

Here, we will calculate 3 types of Average Recovery based on Time Series
Why take Average? Ans: to Reduce Bias, to Increase Consistency

Average_Recovered_April -> Average Recovery Cases for month of April

Average_Recovered_4/1-5/11 -> Average Recovery Cases from 4/1/2020 to 5/11/2020

Average_Recovered_5/1-5/11 -> Average Recovery Cases from 5/1/2020 to 5/11/2020

5/1-5/11 is much Shorter period and more Recent

in analysis part, we will see that 5/1-5/11 has HIGHEST Recovery cases with Protective Factors

why? -> a) Protective Measures are working effectively; b) as time goes by, we have more knowledge and experience of how to fight against COVID-19

In [38]:
#36
recover_inpatient_measures['Average_Recovered_April'] = recover_inpatient_measures.iloc[:,31:61].mean(axis=1)
recover_inpatient_measures['Average_Recovered_4/1-5/11'] = recover_inpatient_measures.iloc[:,31:72].mean(axis=1)
recover_inpatient_measures['Average_Recovered_5/1-5/11'] = recover_inpatient_measures.iloc[:,61:72].mean(axis=1)

-> We will group both transfers to ICU and superior hospital into 1 category

-> Name this category as 'Transfer to ICU or superior hospital

Total of 3 categories in 'Clinical outcome' -> ('Improved and discharge', 'Died', 'Transfer to ICU or superior hospital')

In [39]:
#37
recover_inpatient_measures['Clinical outcome']=recover_inpatient_measures['Clinical outcome'].map({
'Improved and discharge': 'Improved and discharge',
'Died': 'Died',
'Aggravated,turn to ICU': 'Transfer to ICU or superior hospital',
'Aggravated,transfer to superior hospital': 'Transfer to ICU or superior hospital'
})

Let's check to make sure for 'Clinical outcome' feature

In [40]:
#38
recover_inpatient_measures['Clinical outcome'].value_counts().sort_values(ascending=False)

Improved and discharge                  2677031
Transfer to ICU or superior hospital     126698
Died                                      27048
Name: Clinical outcome, dtype: int64

We want to focus on Recovery here; thus, we will exclude 'Died';
Let's only limit to 2 types of patients from 'Clinical outcome' feature:

-> 1. Improved and discharge (recovery after hospitalization)

-> 2. Transfer to ICU or superior hospital (recovery after ICU admission)

In [41]:
#39
recover_inpatient_measures = recover_inpatient_measures[recover_inpatient_measures['Clinical outcome']!='Died']
print(recover_inpatient_measures.shape)

(2803729, 82)


### Part-3: Data+Stats Analysis (#40-#69); Features to be used

-> Comorbidities (Major Risk Factor)

-> Clinical outcome (Comparison among Recovery cases)

-> top6_protective_factors_1 (to analyze how Protective Factors help Increase recovery)

-> who_protective_factors_1 (to analyze how WHO Protective Factors help Increase recovery)

-> Impact comparison between WHO Protective Factors and normal Protective Factors

-> Average_Recovered_April (Comparison among Recovery cases)

-> Average_Recovered_4/1-5/11 (Comparison among Recovery cases)

-> Average_Recovered_5/1-5/11 (Comparison among Recovery cases)

a) Comorbidities+Clinical Outcome, how both together can impact recovery?

-> Just to see Average Recovery based on only these two features, 

In [42]:
#40
comorbid_outcome = {
    #'Latestrecovered_5/11/20': ['mean'],
    'Average_Recovered_April': ['mean'],
    'Average_Recovered_4/1-5/11': ['mean'],
    'Average_Recovered_5/1-5/11': ['mean'],
}
comorbid_outcome = recover_inpatient_measures.groupby(['Comorbidities', 'Clinical outcome']).agg(comorbid_outcome).round().astype(int)
comorbid_outcome.columns = ['Type_' + ('_'.join(col).strip()) for col in comorbid_outcome.columns.values]
comorbid_outcome.reset_index(inplace=True)

In [43]:
#41
comorbid_outcome

,Comorbidities,Clinical outcome,Type_Average_Recovered_April_mean,Type_Average_Recovered_4/1-5/11_mean,Type_Average_Recovered_5/1-5/11_mean
0,N,Improved and discharge,3333,3815,5130
1,N,Transfer to ICU or superior hospital,2440,2521,2743
2,Y,Improved and discharge,3382,3886,5261
3,Y,Transfer to ICU or superior hospital,2440,2521,2743


For Yes to Comorbidities, Average Recovered from 4/1/20 to 5/11/20

-> Recovery cases after admission to ICU < after only hospitalization (Improved/Discharged)
Comorbidities | Hospitalization/ICU	| Type_Average_Recovered_4/1-5/11_mean 
--- | --- | --- 
Y | Improved/Discharged | 3886
Y |	ICU	| 2521

b) Can add-in Protective Factors Increase Recovery for patients with comorbidities compared to w/o effective measures?

Ans: Yes, Protective Factors Increase Recovery in both cases (hospitalization, ICU)
-> Please refer to following blocks of codes and explanations for comparison

In [44]:
#42 #Cormobidities
from scipy import stats
yes_Comorb = recover_inpatient_measures[recover_inpatient_measures['Comorbidities']=='Y']
no_Comorb = recover_inpatient_measures[recover_inpatient_measures['Comorbidities']=='N']

In [45]:
#43
from scipy.stats import ttest_ind
t_statistics = []
p_value = []
for i in recover_inpatient_measures.columns[-2:]:
    t= ttest_ind(yes_Comorb[i],no_Comorb[i])[0]
    t_statistics.append(t)
    p = ttest_ind(yes_Comorb[i],no_Comorb[i])[1]
    p_value.append(p)

In [46]:
#44
t_statistics

[4.803072751124694, 6.718940857034333]

In [47]:
#45
p_value 

[1.562571293315382e-06, 1.8308490862834556e-11]

In [48]:
#46
Comorb_p = pd.DataFrame(data={'Comorbidities_p_value':p_value}, 
                          index=recover_inpatient_measures.columns[-2:])

In [49]:
#47
Comorb_p

,Comorbidities_p_value
Average_Recovered_4/1-5/11,1.562571e-06
Average_Recovered_5/1-5/11,1.830849e-11


In [50]:
#48 #Public health measures
from scipy import stats
yes_PHM = recover_inpatient_measures[recover_inpatient_measures['who_protective_factors']=='Public health measures']
no_PHM = recover_inpatient_measures[recover_inpatient_measures['who_protective_factors']!='Public health measures']

In [51]:
#49
from scipy.stats import ttest_ind
t_statistics = []
p_value = []
for i in recover_inpatient_measures.columns[-2:]:
    t= ttest_ind(yes_PHM[i],no_PHM[i])[0]
    t_statistics.append(t)
    p = ttest_ind(yes_PHM[i],no_PHM[i])[1]
    p_value.append(p)

In [52]:
#50
t_statistics

[70.75507172456459, 91.86278540852962]

In [53]:
#51
p_value

[0.0, 0.0]

In [54]:
#52
PHM_p = pd.DataFrame(data={'PublicHealthMeasures_p_value':p_value}, 
                          index=recover_inpatient_measures.columns[-2:])

In [55]:
#53
PHM_p

,PublicHealthMeasures_p_value
Average_Recovered_4/1-5/11,0.0
Average_Recovered_5/1-5/11,0.0


In [56]:
#54 #Special populations
from scipy import stats
yes_SP = recover_inpatient_measures[recover_inpatient_measures['who_protective_factors']=='Special populations']
no_SP = recover_inpatient_measures[recover_inpatient_measures['who_protective_factors']!='Special populations']

In [57]:
#55
from scipy.stats import ttest_ind
t_statistics = []
p_value = []
for i in recover_inpatient_measures.columns[-2:]:
    t= ttest_ind(yes_SP[i],no_SP[i])[0]
    t_statistics.append(t)
    p = ttest_ind(yes_SP[i],no_SP[i])[1]
    p_value.append(p)

In [58]:
#56
t_statistics

[21.35577113513604, 27.090876035848027]

In [59]:
#57
p_value

[3.51115747697295e-101, 1.3234237158547195e-161]

In [60]:
#58
SP_p = pd.DataFrame(data={'SpecialPopulation_p_value':p_value}, 
                          index=recover_inpatient_measures.columns[-2:])

In [61]:
#59
SP_p

,SpecialPopulation_p_value
Average_Recovered_4/1-5/11,3.511157e-101
Average_Recovered_5/1-5/11,1.323424e-161


In [62]:
#60 #Domestic_travel
from scipy import stats
yes_DT = recover_inpatient_measures[recover_inpatient_measures['who_protective_factors']=='Domestic travel']
no_DT = recover_inpatient_measures[recover_inpatient_measures['who_protective_factors']!='Domestic travel']

In [63]:
#61
from scipy.stats import ttest_ind
t_statistics = []
p_value = []
for i in recover_inpatient_measures.columns[-2:]:
    t= ttest_ind(yes_DT[i],no_DT[i])[0]
    t_statistics.append(t)
    p = ttest_ind(yes_DT[i],no_DT[i])[1]
    p_value.append(p)

In [64]:
#62
t_statistics

[41.71670159257347, 61.45214299423668]

In [65]:
#63
p_value

[0.0, 0.0]

In [66]:
#64
DT_p = pd.DataFrame(data={'DomesticTravel_p_value':p_value}, 
                          index=recover_inpatient_measures.columns[-2:])

In [67]:
#65
DT_p

,DomesticTravel_p_value
Average_Recovered_4/1-5/11,0.0
Average_Recovered_5/1-5/11,0.0


In [68]:
#66
protect_p_value = PHM_p.join(SP_p).join(DT_p)

In [69]:
#67
protect_p_value

,PublicHealthMeasures_p_value,SpecialPopulation_p_value,DomesticTravel_p_value
Average_Recovered_4/1-5/11,0.0,3.511157e-101,0.0
Average_Recovered_5/1-5/11,0.0,1.323424e-161,0.0


In [70]:
#68
whoprotect_comorbid_outcome = {
    #'Latestrecovered_5/11/20': ['mean'],
    'Average_Recovered_April': ['mean'],
    'Average_Recovered_4/1-5/11': ['mean'],
    'Average_Recovered_5/1-5/11': ['mean'],
}
whoprotect_comorbid_outcome = recover_inpatient_measures.groupby(['Comorbidities', 'Clinical outcome', 'who_protective_factors']).agg(whoprotect_comorbid_outcome).round().astype(int)
whoprotect_comorbid_outcome.columns = ['Type_' + ('_'.join(col).strip()) for col in whoprotect_comorbid_outcome.columns.values]
whoprotect_comorbid_outcome.reset_index(inplace=True)

In [71]:
#69
whoprotect_comorbid_outcome.tail(11) #who_protective_factors

,Comorbidities,Clinical outcome,who_protective_factors,Type_Average_Recovered_April_mean,Type_Average_Recovered_4/1-5/11_mean,Type_Average_Recovered_5/1-5/11_mean
11,Y,Improved and discharge,Communications and engagement,3161,3576,4706
12,Y,Improved and discharge,Detecting and isolating cases,3300,3861,5392
13,Y,Improved and discharge,Domestic travel,4906,6206,9751
14,Y,Improved and discharge,Not of interest,324,622,1436
15,Y,Improved and discharge,Public health measures,4652,5602,8192
16,Y,Improved and discharge,Special populations,5304,6436,9523
17,Y,Transfer to ICU or superior hospital,Communications and engagement,2427,2494,2677
18,Y,Transfer to ICU or superior hospital,Detecting and isolating cases,2254,2328,2531
19,Y,Transfer to ICU or superior hospital,Domestic travel,2534,2878,3817
20,Y,Transfer to ICU or superior hospital,Public health measures,2618,2784,3237


Top3 WHO Protective Facotrs' Positive Impacts on Recovery (w/ Comorbidities, 4/1-5/11)

Comorbidities |	Hospitalization/ICU | AvgRecovery Cases w/o	| AvgRecovery w/ Special Populations target	| AvgRecovery w/ Domestic travel restriction | AvgRecovery w/ Public health measures
--- | --- | --- | --- |--- |---
Y |	Improved/Discharged | 3886 | 6436 |	6206 | 5602
Y |	ICU | 2521 | 3024 |	2878 | 2784

-> Recovery cases w/ WHO Protective Factors > w/ Country-Level Protective Factors

Recovery has improved in both cases with WHO Protective Factors

Apparently, recovery after hospitalization has greater improvement than 
recovery after ICU admission and there exist disparities in effectiveness of protective factors

4. Overall Best Protective Factors to prevent/reduce risks:

a) Public health measures

b) Special populations

c) Domestic travel_Social measures (Travel and Movement Restructions)

d) Quarantine / Monitoring